In [3]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import sys
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
from sklearn.metrics import confusion_matrix
from ml_util import *
from sklearn.model_selection import cross_val_score

cols = ['UpdateTime', 'UpdateMillisec', 'InstrumentID', 'Volume', 'LastPrice', 'OpenInterest', 'PreSettlementPrice',
        'PreClosePrice', 'PreOpenInterest', 'OpenPrice', 'HighestPrice', 'LowestPrice', 'ClosePrice', 'SettlementPrice',
        'UpperLimitPrice', 'LowerLimitPrice', 'BidPrice1', 'AskPrice1', 'BidVolume1', 'AskVolume1', 'Turnover', 'TradingDay', 'LocalTime']

def MACD(mp, window_size):
    ema_long = mp.ewm(span=2*window_size, adjust=False).mean()
    ema_short = mp.ewm(span=window_size, adjust=False).mean()
    diff = ema_long - ema_short
    dea = diff.ewm(span=math.ceil(0.8*window_size), adjust=False).mean()
    return 2*(diff - dea)

contract_size = 300
time_set = [1,3,5,10,20,30,50, 100, 200, 500]
x_col = []
y_col = 'yield_diff'
df_list = []
for f in os.listdir('.'):
    if 'tick' not in f:
        continue
    df = pd.read_csv(f, header=0, names = cols, dtype={'UpdateTime':str, 'UpdateMillisec':str})
    df['mp'] = (df['BidPrice1'] + df['AskPrice1'])/2
    df['wp'] = (df['BidPrice1'] * df['BidVolume1'] + df['AskPrice1'] * df['AskVolume1']) / (df['BidVolume1'] + df['AskVolume1'])
    df['rwp'] = (df['BidPrice1'] * df['AskVolume1'] + df['AskPrice1'] * df['BidVolume1']) / (df['BidVolume1'] + df['AskVolume1'])
    df['atp'] = df['Turnover'].diff(1) / df['Volume'].diff(1) / contract_size
    df['wp-mp'] = df['wp'] - df['mp']
    df['rwp-mp'] = df['rwp'] - df['mp']
    df['atp-mp'] = df['atp'] - df['mp']
    df['r10'] = df['mp'].diff(-10) / df['mp']
    df = df[df['UpdateTime'] > '09']
    df['time_str'] = df['UpdateTime'] + df['UpdateMillisec']
    df_x = df[df['InstrumentID'] == 'IF2009']
    df_y = df[df['InstrumentID'] == 'IF2008']
    df = pd.merge(df_x, df_y, on=['time_str'], how='inner')
    df['mid_delta'] = df['mp_x'] - df['mp_y']
    for t in time_set:
        for s in ['x', 'y']:
            df['mid_diff_%s_%d' % (s, t)] = df['mp_%s' % (s)].diff(t)
            x_col.append('mid_diff_%s_%d' % (s, t))
            df['openinterest_diff_%s_%d' % (s, t)] = df['OpenInterest_%s' % (s)].diff(t)
            x_col.append('openinterest_diff_%s_%d' % (s, t))
            df['mid_delta_diff_%d' % (t)] = df['mid_delta'].diff(t)
            x_col.append('mid_delta_diff_%d' % (t))
            df['macd_mid_%s_%d' % (s, t)] = MACD(df['mp_%s'%(s)], t)
            x_col.append('macd_mid_%s_%d' % (s, t))
            df['wp-mp_diff_%s_%d' % (s, t)] = df['wp-mp_%s' %(s)].diff(t)
            x_col.append('wp-mp_diff_%s_%d' % (s, t))
            df['atp-mp_diff_%s_%d' % (s, t)] = df['atp-mp_%s' %(s)].diff(t)
            x_col.append('atp-mp_diff_%s_%d' % (s, t))
            df['rwp-mp_diff_%s_%d' % (s, t)] = df['rwp-mp_%s' %(s)].diff(t)
            x_col.append('rwp-mp_diff_%s_%d' % (s, t))
        if t > 1:
            df['mid_delta_gap_%d'%(t)] = (df['mid_delta'] - df['mid_delta'].rolling(t).mean())/df['mid_delta'].rolling(t).std()
            x_col.append('mid_delta_gap_%d'%(t))
    df = df[~np.isnan(df['r10_x'])]
    df_list.append(df)

x_col += ['wp-mp_x', 'atp-mp_x', 'rwp-mp_x']
x_col += ['wp-mp_y', 'atp-mp_y', 'rwp-mp_y']
    
df = reduce(lambda x,y: x.append(y), df_list)
df[y_col] = df['r10_x'] - df['r10_y']
#df = df.fillna(0.0)
x, y = df[x_col].values, df[y_col].values

x = Imputer(missing_values='NaN', strategy='mean',axis=0).fit_transform(x)
x = RobustScaler().fit_transform(x)
x_col_map = {t:i for i, t in enumerate(x_col)} 

In [ ]:
plt.title('y distribution')
plt.hist(df[y_col])
plt.show()

In [ ]:
PlotFactorCorr(x, y)

In [4]:
# split date into training_set and test_set
from sklearn.preprocessing import OneHotEncoder
code = np.reshape((y>0), (-1, 1))
y= OneHotEncoder().fit(code).transform(code).toarray()
test_ratio = 0.2
valid_ratio = 0.2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_ratio, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state=0)

y.shape

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in greater
  This is separate from the ipykernel package so we can avoid doing imports until


(125395, 2)

In [ ]:
# linear model
#model = linear_model.ElasticNetCV(alphas=[0.0,0.0001, 0.0005, 0.001, 0.01, 0.1, 1, 10], l1_ratio=[0.0, .01, .1, .5, .9, .99, 0.1],  max_iter=5000,  tol=1e-15)
#model.fit(x_train, y_train)
#print('expr:y=%s*x+%lf, iter=%d' % (str(model.coef_), model.intercept_, model.n_iter_))

In [ ]:
# xgboost
model = xgb.XGBClassifier(max_depth=50, learning_rate=0.1, n_estimators=1000, silent=False, objective='binary:logistic', num_classes=2,
                          booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0,
                          subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1)
eval_set = [(x_train, y_train>0),(x_valid, y_valid>0)]
model.fit(x_train, y_train>0, eval_set=eval_set, verbose=True, eval_metric='logloss', early_stopping_rounds=10)
y_pred = model.predict(x_test)

In [ ]:
a = ClsReport(model, y_train>0, model.predict(x_train)>0, binary=True)
selected_features = [x_col_map[i] for i in a]
x = x[:, a]
test_ratio = 0.2
valid_ratio = 0.2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_ratio, random_state=0)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state=0)

In [ ]:
#from sklearn.model_selection import cross_val_score
#a = ClsReport(model, y_test>0, y_pred>0, binary=True)
#model = xgb.XGBClassifier(max_depth=50, learning_rate=0.1, n_estimators=1000, silent=False, objective='binary:logistic', num_classes=2,
                          #booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0,
                          #subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1)
#cross_val_score(model, x, y>0, cv=5)

In [6]:
from numpy import random
def GetSamples(x, y, batch_size=64, seq_length=32):
    count = 0
    rx, ry, index = [], [], []
    while count < batch_size:
        i = random.randint(len(x)-seq_length) #, size=batch_size)
        tx, ty = x[i:i+seq_length], y[i+seq_length-1]
        if np.isnan(tx).sum() != 0:
            continue
        rx.append(tx)
        ry.append(ty)
        index.append(i)
        count += 1
    return np.array(rx), np.array(ry), np.array(index)

In [7]:
xt, yt, _ = GetSamples(x_train, y_train)#, 1, 1)
xt.shape, yt.shape
#np.isnan(xt).sum()

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras import activations

batch_size = 128
seq_length = 32
def lstm_model():
    model = keras.Sequential()
    model.add(layers.Dense(300, activation='relu', input_shape=(seq_length, x.shape[1]))),#, return_sequences=True))
    model.add(layers.Flatten())
    #model.add(layers.LSTM(2, return_sequences=False,name='out'))
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(2, name='out'))
    model.add(layers.Activation('sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-2),
                 loss=keras.losses.CategoricalCrossentropy(),
                 metrics=['accuracy'])
    return model

model = lstm_model()
model.summary()

#mid_model = Model(inputs=model.input, outputs=model.get_layer('out').output)


In [ ]:
num = 1#batch_size
xt, yt, index = GetSamples(x_train, y_train, batch_size=batch_size, seq_length=seq_length)
for i in range(100):
    #print(np.isnan(xt).sum())
    print(model.predict(xt[:num]), GetMidRes(model, 'out', xt[:num]))
    model.fit(xt[:num], yt[:num])
    #print(model.predict(xt[:num]), yt[:num], GetMidRes(model, 'out', xt[:num]))
    #if np.isnan(model.predict(xt[:num])).sum() > 0:
    print(xt[:num], yt[:num], index, model.predict(xt[:num]))
    raw_input()
    #break

In [ ]:
for col in x_col:
    df[col].describe()
    df[col].plot.box(title=col)
    plt.show()
    

In [ ]:
from sklearn.preprocessing import StandardScaler as ss
from sklearn.preprocessing import Imputer
df['atp-mp_diff_x_20'].plot.box()
plt.show()
d = np.reshape(df['atp-mp_diff_x_20'].tolist(), (-1, 1))
d = Imputer(missing_values='NaN', strategy='mean',axis=0).fit_transform(d)
plt.boxplot(ss().fit_transform(d))
plt.show()
plt.hist(ss().fit_transform(d))
plt.show()
ss().fit_transform(x)